In [15]:
import sim
import numpy as np
import time as t
import matplotlib.pyplot as plt 
import cv2
import random as r

def connect(port):
# Establece la conexión a VREP
# port debe coincidir con el puerto de conexión en VREP
# retorna el número de cliente o -1 si no puede establecer conexión
    sim.simxFinish(-1) # just in case, close all opened connections
    clientID=sim.simxStart('127.0.0.1',port,True,True,2000,5) # Conectarse
    if clientID == 0: print("conectado a", port)
    else: print("no se pudo conectar")
    return clientID

#### .... y movemos el robot
Utilizando los manejadores, podemos enviar parámetros a los elementos


In [23]:
clientID = connect(20000)
#obtenemos los manejadores para cada una de las articulaciones del robot
ret, joint0 = sim.simxGetObjectHandle(clientID, 'joint_0',sim.simx_opmode_blocking)
ret, joint1 = sim.simxGetObjectHandle(clientID, 'joint_1',sim.simx_opmode_blocking)
ret, joint2 = sim.simxGetObjectHandle(clientID, 'joint_2',sim.simx_opmode_blocking)
ret, joint3 = sim.simxGetObjectHandle(clientID, 'joint_3',sim.simx_opmode_blocking)

ret, tip = sim.simxGetObjectHandle(clientID, 'suctionPad',sim.simx_opmode_blocking)

ret, sensorHandle = sim.simxGetObjectHandle(clientID, 'Vision_sensor',sim.simx_opmode_blocking)

conectado a 20000


In [17]:
velocidad_de_movimiento = 7
vm = velocidad_de_movimiento

def set_ang(join_n, ang):
    sim.simxSetJointTargetPosition(clientID, join_n, np.deg2rad(ang), sim.simx_opmode_oneshot)

def set_angles(angs):
    set_ang(joint0, angs[0])
    set_ang(joint1, angs[1])   
    set_ang(joint2, angs[2])   
    set_ang(joint3, angs[3])

In [18]:
class robot:
    def __init__(self, ang0, ang1, ang2, ang3, delta):
        self.ang0 = ang0
        self.ang1 = ang1
        self.ang2 = ang2
        self.ang3 = ang3
        self.delta = delta
        
    def setEffector(self,val):
        # acciona el efector final
        # val es Int con valor 0 ó 1 para desactivar o activar el actuador final.
        res,retInts,retFloats,retStrings,retBuffer=sim.simxCallScriptFunction(clientID,
        "suctionPad", sim.sim_scripttype_childscript,"setEffector",[val],[],[],"", sim.simx_opmode_blocking)
        return res
    
    
    def ang2_var(self, new_ang2):
        dif_ang2 = self.ang2 - new_ang2
        self.ang2 = new_ang2
        self.ang3 = self.ang3 + dif_ang2
        
        
    def ang1_var(self, new_ang1):
        dif_ang1 = self.ang1 - new_ang1
        self.ang2 = self.ang2 + dif_ang1
        self.ang1 = new_ang1
  
     
    def get_angles(self):
        return [self.ang0, self.ang1, self.ang2, self.ang3]

    
    def camino_ang0(self, new_ang0):
        neg_pos = -1 if new_ang0<=0 else 1
        array = np.arange(self.ang0, new_ang0, self.delta*neg_pos)
        for ind,i in enumerate(array):
            set_ang(joint0, i)
            if ind%vm ==0: t.sleep(0.01)
        
        self.ang0 = new_ang0
    
    
    
    def camino_ang1(self, new_ang1):      
        neg_pos = -1 if self.ang1>new_ang1 else 1
        array = np.arange(self.ang1, new_ang1, self.delta*neg_pos)
        for ind,i in enumerate(array):
            if ind%vm ==0: t.sleep(0.01)
            self.ang1_var(i)
            set_angles(self.get_angles())
            
    
    
    def camino_ang2(self, new_ang2):  
        neg_pos = -1 if self.ang2>new_ang2 else 1
        array = np.arange(self.ang2, new_ang2, self.delta*neg_pos)
        for ind,i in enumerate(array):
            if ind%vm ==0: t.sleep(0.01)
            self.ang2_var(i)
            set_angles(self.get_angles())
               
    


robot = robot(0, 90, -90, -90, .2)
set_angles(robot.get_angles())



#pide la imagen del sensor y envia el promedio de los colores
def prom_R_G_B():
    retCode, resolution, image = sim.simxGetVisionSensorImage(clientID, sensorHandle, 0, sim.simx_opmode_oneshot_wait)
    img = np.array(image,dtype= np.uint8)
    img.resize([resolution[1], resolution[0], 3])
    
        
    rojo = np.mean(img[:,:,0])
    verde = np.mean(img[:,:,1])
    azul = np.mean(img[:,:,2])
    return [rojo, verde, azul]


def color_selector():
    G = [95, 60, -40]
    R =  [-95, 50, -40]
    B = [-180, 60, -40]
    rojo, verde, azul = prom_R_G_B()
    if(rojo> 200): return R
    if(azul> 200): return B
    if(verde> 200): return G
    return [0,90, 0]
            
        

        

In [19]:
#posicion de los cubos
angs =[
    [10, -1.8, 0],
    [0, 2.8, -10],
    [-10, 2.3, -10],
    
    [0, 30, -65.4],
    [10, 30, -65.4],
    [-10, 30, -65.4]
]




def clasificador(v):
    robot.camino_ang2(v[2])  
    robot.camino_ang1(v[1])  
    robot.camino_ang0(v[0])
    robot.setEffector(0)
    
def elector_de_cubo(angs):
    robot.camino_ang0(angs[0])
    robot.camino_ang1(angs[1])
    robot.camino_ang2(angs[2])    
    robot.setEffector(1)#succion    

    
def ingreso_de_un_cubo(angs):
    elector_de_cubo(angs)   
    #constante a todos
    robot.camino_ang1(60) 
    robot.camino_ang2(-40) 
    clasificador(color_selector())    

    
    



In [24]:
for i in r.sample(range(6), 6):    
    ingreso_de_un_cubo(angs[i])